In [1]:
# get spark session, 2g mem per executor
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os

# set python env
os.environ['PYSPARK_PYTHON'] = "/opt/conda3/envs/lab2/bin/python"
spark = SparkSession.builder \
    .appName("item_list_to_redis") \
    .master("spark://node01:10077") \
    .enableHiveSupport()\
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.cores.max", "2") \
    .config("spark.sql.shuffle.partitions", "12") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

sc = spark.sparkContext

23/04/23 02:02:52 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/23 02:02:53 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/23 02:02:53 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/04/23 02:02:53 WARN spark.SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [2]:
import redis
def send_hot_list(item):
    pool = redis.ConnectionPool(host='cowstudio.wayne-lee.cn',port=3002,password='cowstudio', decode_responses=True)
    redis_cli = redis.Redis(connection_pool=pool)
    item_id,category,score,rn,date = item
    redis_cli.zadd("item-hot-list-"+category,{item_id:score})
    return f"item-hot-list-{category}",item_id,score
def send_fresh_list(item):
    pool = redis.ConnectionPool(host='cowstudio.wayne-lee.cn',port=3002,password='cowstudio', decode_responses=True)
    redis_cli = redis.Redis(connection_pool=pool)
    item_id,category,score,rn,date = item
    redis_cli.zadd("item-fresh-list-"+category,{item_id:score})
    return f"item-fresh-list-{category}",item_id,score

In [3]:
# define map functions 
from datetime import datetime, timedelta

today_string = datetime.today().strftime('%Y-%m-%d')
print(today_string)

2023-04-23


In [4]:
item_hot_list = spark.sql(f'''
select
    *
from
    item_hot_list
where
    date = '{today_string}'
''').rdd

print(item_hot_list.take(1))
item_hot_list.map(send_hot_list).collect()

23/04/23 02:03:07 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


[]


[]

In [5]:
item_fresh_list = spark.sql(f'''
select
    *
from
    item_fresh_list
where
    date = '{today_string}'
''').rdd

print(item_fresh_list.take(1))
item_fresh_list.map(send_fresh_list).collect().task(10)

[Row(item_id=285, category='service', score=0.0, rn=88, date='2023-04-23')]


AttributeError: 'list' object has no attribute 'task'

In [ ]:
spark.stop()